<a href="https://colab.research.google.com/github/alisonsoong/NASA-SEES-Internship-2021/blob/main/Night_Light_and_PopGrid_example_Soong_Alison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/notebooks/geemap_and_earthengine.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/geemap_and_earthengine.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

Example of VIIRS (2012-present, monthly) and DMSP (1992-2014) night light datasets + Population Grid dataset

— Alison

In [2]:
import ee # Import Earth Engine API

In [3]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=VSUdfKTjJhXV4Bw_fdldPteU66PBabOWyOxJUmg7J10&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhrX3H6d-EubsQPbb3PBqU9Wpln9afvSzvlZ_oEDO35gU6qAeWuZNs

Successfully saved authorization token.


In [4]:
# Installs geemap package
import subprocess

try:
    import geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap
    print("geemap is now installed, imported and ready to use in Colab") 


geemap package not installed. Installing ...
geemap is now installed, imported and ready to use in Colab


In [65]:
# VIIRS
collectionNight2019  = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG') \
    .filter(ee.Filter.date('2019-01-01', '2019-12-31'))

collectionNight2020  = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG') \
    .filter(ee.Filter.date('2020-01-01', '2020-12-31'))

night_light_params = {
    'bands': ['avg_rad'],
    'min': 0, 
    'max': 30,
}

imageNight2019 = collectionNight2019.mean()
imageNight2020 = collectionNight2020.mean()

In [112]:
# DMSP
collectionDMSP1992 = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS") \
    .filter(ee.Filter.date('1992-01-01', '1992-12-31'))

collectionDMSP2013 = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS") \
    .filter(ee.Filter.date('2013-01-01', '2013-12-31'))

imageDMSP1992 = collectionDMSP1992.select('stable_lights').mean()
imageDMSP2013 = collectionDMSP2013.select('stable_lights').mean()

dmsp_params = {
  'min': 0.0,
  'max': 63.0,
  'gamma' : 1.0
}

In [42]:
# population grid
dataset2020 = ee.ImageCollection("WorldPop/GP/100m/pop_age_sex_cons_unadj")\
    .filterDate("2020-01-01", "2020-12-01")

pop_params = {
  'bands': ['population'],
  'min': 0.0,
  'max': 50.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
}

In [114]:
roi = ee.Geometry.Point(-122.438005, 37.729844) # (long, lat)

Map = geemap.Map()
Map.addLayer(imageNight2019, night_light_params, "VIIRS Night image: 2019", False)
Map.addLayer(imageNight2020, night_light_params, "VIIRS Night image: 2020", False)
Map.addLayer(dataset2020, pop_params, "Population 2020", False);
Map.addLayer(imageDMSP1992, dmsp_params, "DMSP Night image: 1992", False)
Map.addLayer(imageDMSP2013, dmsp_params, "DMSP Night image: 2013")
Map.centerObject(roi, 6)
Map
